In [1]:
import os
import pickle
import numpy as np

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


In [3]:
def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

def load_cifar100_data(data_dir):
    # Load training data
    train_file = os.path.join(data_dir, 'train')
    train_data_dict = unpickle(train_file)
    x_train = train_data_dict[b'data']
    y_train = train_data_dict[b'fine_labels']  # or b'coarse_labels' for coarse labels

    # Load testing data
    test_file = os.path.join(data_dir, 'test')
    test_data_dict = unpickle(test_file)
    x_test = test_data_dict[b'data']
    y_test = test_data_dict[b'fine_labels']  # or b'coarse_labels' for coarse labels

    # Load meta data
    meta_file = os.path.join(data_dir, 'meta')
    meta_data_dict = unpickle(meta_file)
    label_names = meta_data_dict[b'fine_label_names']  # or b'coarse_label_names'

    # Reshape and convert to grayscale
    x_train = x_train.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    x_test = x_test.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    x_train = np.array([rgb2gray(img) for img in x_train])
    x_test = np.array([rgb2gray(img) for img in x_test])

    x_train = x_train.astype('uint8')
    x_test = x_test.astype('uint8')
    return (x_train, y_train), (x_test, y_test), label_names


In [4]:
!pip install xgboost

In [5]:
from util import Utils

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [6]:
(cifar_x_train, cifar_y_train), (cifar_x_test, cifar_y_test), label_names = load_cifar100_data('cifar-100-python')

In [7]:
# Reshape the data: CIFAR-100 images are 32x32 pixels with 3 color channels (RGB)
cifar_x_train = cifar_x_train.reshape(-1, 32 * 32)
cifar_x_test = cifar_x_test.reshape(-1, 32 * 32)

# Normalize pixel values
cifar_x_train, cifar_x_test = cifar_x_train / 255.0, cifar_x_test / 255

In [8]:
cifar_x_train = cifar_x_train.astype('float32')
cifar_x_test = cifar_x_test.astype('float32')

In [9]:
from sklearn import model_selection
cifar_X_train, cifar_x_val, cifar_Y_train, cifar_y_val = model_selection.train_test_split(cifar_x_train, cifar_y_train, test_size=1/3, random_state=42)


In [12]:
best_cifar_model = Utils.find_best_rf_parameters(cifar_x_train, cifar_y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/home/ahmad/anaconda3/envs/rapids-23.12/lib/python3.10/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



All Results:
{'n_estimators': 100, 'max_features': 'log2', 'max_depth': 100, 'criterion': 'entropy'} with accuracy: 0.13742006811129828

--- 304.1415596008301 seconds ---
The highest Accuracy 0.13742 is the model with parameters: {'n_estimators': 100, 'max_features': 'log2', 'max_depth': 100, 'criterion': 'entropy'}
[CV] END criterion=entropy, max_depth=100, max_features=log2, n_estimators=100; total time= 2.1min
[CV] END criterion=entropy, max_depth=100, max_features=log2, n_estimators=100; total time= 2.1min
[CV] END criterion=entropy, max_depth=100, max_features=log2, n_estimators=100; total time= 2.1min


In [ ]:
Utils.accuracy_measure_rf(cifar_x_train, cifar_y_train, 
                          cifar_x_test, cifar_y_test, 
                          n_estimators = 100, criterion = 'gini', max_depth = 50, max_features = 'log2')

In [ ]:
best_xgb_model = Utils.find_best_xgb_parameters(cifar_X_train, cifar_Y_train, cifar_x_val, cifar_y_val)

In [11]:
Utils.accuracy_measure_xgb(cifar_x_train, cifar_y_train, 
                          cifar_x_test, cifar_y_test, 
                           n_estimators = 100, learning_rate = 0.1, max_depth = 50, subsample = 0.8, colsample_bytree = 1)

--- 5639.89879655838 seconds ---


(0.1917,
 (array([0.40659341, 0.23780488, 0.10989011, 0.06666667, 0.13461538,
         0.10447761, 0.1025641 , 0.1512605 , 0.28358209, 0.34375   ,
         0.07042254, 0.22033898, 0.13541667, 0.20689655, 0.11235955,
         0.12820513, 0.33333333, 0.29565217, 0.04761905, 0.1097561 ,
         0.45736434, 0.23121387, 0.40983607, 0.19871795, 0.38356164,
         0.20754717, 0.26666667, 0.07692308, 0.44230769, 0.34920635,
         0.09375   , 0.0990099 , 0.2027027 , 0.12751678, 0.07246377,
         0.12345679, 0.20869565, 0.11363636, 0.11805556, 0.30769231,
         0.25882353, 0.53191489, 0.16964286, 0.08235294, 0.07843137,
         0.08139535, 0.0989011 , 0.192     , 0.41176471, 0.18320611,
         0.06896552, 0.10071942, 0.2962963 , 0.21238938, 0.1637931 ,
         0.07407407, 0.15328467, 0.19736842, 0.24742268, 0.16161616,
         0.27642276, 0.42424242, 0.09090909, 0.16037736, 0.11881188,
         0.05633803, 0.12162162, 0.09433962, 0.27516779, 0.22142857,
         0.1       , 0.24

In [ ]:
print("a")